<a href="https://colab.research.google.com/github/yohan-th/21sh/blob/master/Scrapping_senscritique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import sys
import csv
import threading
import time
import urllib.request


# Parametre
debug = False
save_tmp_out = True

# **Tools**

In [ ]:
def get_html(url:str, max_attempt:int):
  if debug == True: print(url)
  attempt = 1

  while (attempt <= max_attempt):
    try:
      with urllib.request.urlopen(url) as response:
        html = response.read().decode('utf-8')
        return html
    except OSError as e:
      print(f"[Error {e.code}] {e.reason} : {url}")
      if e.code == 503:
        time.sleep(60)
      time.sleep(1)
      attempt += 1
  return -1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def save_file(f, data):
    with open('/content/gdrive/My Drive/Colab Notebooks/senscritique/'+f, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

# Script

Récupération des pages tops films sur senscritique

In [ ]:
class GetAllTops_Thread(threading.Thread):

    def __init__(self, url, page):
        threading.Thread.__init__(self)
        self.url = url
        self.page = page

    def run(self):
        if debug:
          print("[Ouverture page "+ str(page))
        
        html = get_html(self.url, 5)
        if html == -1:
          print(f"[Error get messages] --> abort page {self.url}")
          return
       
        tops = re.findall(r'data-sc-link="(.+?)"', html)
        for new_top in tops:
          fresh = True
          for got_top in all_top_url:
            if new_top == got_top:
              fresh = False
          if fresh == True:
            all_top_url.extend([new_top])


In [ ]:
print("Recherche des top de sens critiques")
html = get_html('https://www.senscritique.com/films/sondages', 3)
if html == -1:
  print('Sens critique not reacheable. EXIT')
max_page = re.search('">...([0-9]+)<',html).group(1)

print(max_page+" page(s) de classement top film")

all_top_url = ['/films/tops/top111', '/films/tops/top100-des-top10']
global all_top_url
threadList = []
for page in range(int(max_page)):
    newthread = GetAllTops_Thread('https://www.senscritique.com/films/sondages/tous/tous/page-'+str(page + 1), page)
    newthread.start()
    threadList.append(newthread)
    
    if int(page) % 25 == 0:
      print(str(page) + " page téléchargées sur " + max_page)
      for curThread in threadList :
        curThread.join()

print("Attente des threads")
for curThread in threadList :
    curThread.join()

print(all_top_url)
print("nb total de top = " + str(len(all_top_url)))

Recherche des top de sens critiques
226 page(s) de classement top film
0 page téléchargées sur 226
25 page téléchargées sur 226
50 page téléchargées sur 226
75 page téléchargées sur 226
100 page téléchargées sur 226
125 page téléchargées sur 226
150 page téléchargées sur 226
175 page téléchargées sur 226
200 page téléchargées sur 226
225 page téléchargées sur 226
Attente des threads
['/films/tops/top111', '/films/tops/top100-des-top10', '/top/resultats/les_meilleurs_films_de_2022/3167299', '/top/resultats/les_meilleurs_films_de_2021/2917616', '/top/resultats/les_meilleurs_films_batman/1056032', '/top/resultats/les_meilleurs_films_de_2020/2582670', '/top/resultats/les_meilleurs_films_de_2019/2301802', '/top/resultats/les_meilleurs_films_d_animation_pixar/429189', '/top/resultats/les_meilleurs_films_francais_de_2022/3170620', '/top/resultats/les_meilleurs_films_francais_de_2021/3081464', '/top/resultats/les_meilleurs_films_dc_comics/873923', '/top/resultats/les_meilleurs_films_de_super_h

Récupération des films de chaque top

In [ ]:
def check_if_new(all_films, link):
    for sublist in all_films:
        if sublist[4] == link:
            return sublist
    return True

In [ ]:
import html as html_convert

class GetFilmData_Thread(threading.Thread):

    def __init__(self, url):
        threading.Thread.__init__(self)
        self.url = url

    def run(self):
        if debug:
          print("[Ouverture de "+ str(self.url))
        
        ghtml = get_html(self.url, 3)
        if ghtml == -1:
          print(f"[Error get messages] --> abort page {self.url}")
          return

        title = re.search('<title>(.*?) - ', html_convert.unescape(ghtml)).group(1)
        
        if re.search('<title>.*?\(([0-9]{4})\)', ghtml):
          date = re.search('<title>.*?\(([0-9]{4})\)', ghtml).group(1)
        else:
          date = "no_release_date"

        if re.search('d-link-reverse', ghtml):
          note = re.search('d-link-reverse">(.*?)<', ghtml).group(1)
        else:
          note = 'not_scored_yet'

        if re.search('duration.*?>', ghtml):
          duree = re.search('duration.*?>(.*?)<', ghtml, re.MULTILINE | re.DOTALL).group(1)
          duree = re.sub(r"[\n\t\s]*", "", duree)
        else:
          duree = 'not_specified'
        
        
        all_films.append([title,date,note,duree,self.url])

Récupération des 2 types de page : [films_resultat](https://www.senscritique.com/top/resultats/les_meilleurs_films_de_2022/3167299) et [films_tops](https://www.senscritique.com/films/tops/top111)

In [ ]:
def films_resultats(html):
    new_film = 0
    threadList = []
    html_films = re.findall('ListItem(.*?)}', html, re.MULTILINE | re.DOTALL)
    for html_film in html_films:
        link = re.search('url":"(.*?)"', html_film).group(1)
        if check_if_new(all_films, link) == True:
            new_film += 1
            newthread = GetFilmData_Thread(link)
            newthread.start()
            threadList.append(newthread)
            time.sleep(0.3)

    for curThread in threadList :
        curThread.join()
    print(str(new_film)+' new film sur '+str(len(html_films))+' checked. TOTAL: '+str(len(all_films)))

def films_tops(html):
    new_film = 0
    threadList = []
    html_films = re.findall('elto-flexible-column(.*?)"elco-baseline"', html, re.MULTILINE | re.DOTALL)
    for html_film in html_films:
        link = 'https://www.senscritique.com'+re.search('href="(.*?)"', html_film).group(1)
        if check_if_new(all_films, link) == True:
            new_film += 1
            newthread = GetFilmData_Thread(link)
            newthread.start()
            threadList.append(newthread)
            time.sleep(0.3)

    for curThread in threadList :
        curThread.join()
    print(str(new_film)+' new film sur '+str(len(html_films))+' checked. TOTAL: '+str(len(all_films)))

In [ ]:
#########
# DEBUG : lorsqu'il trouve pas les éléments de re.search
#########
# url = "https://www.senscritique.com/top/resultats/les_films_les_plus_attendus_de_2022/3099375"
# html = get_html(url, 3)
# html_films = re.findall('ListItem(.*?)}', html, re.MULTILINE | re.DOTALL)
# for html_film in html_films:
#     link = re.search('url":"(.*?)"', html_film).group(1)
#     print(link)
#     ghtml = get_html(link, 3)
#     #print(ghtml)
#     title = re.search('<title>(.*?) - ', html_convert.unescape(ghtml)).group(1)
#     if re.search('<title>.*?\(([0-9]{4})\)', ghtml):
#       date = re.search('<title>.*?\(([0-9]{4})\)', ghtml).group(1)
#     else:
#       date = "no_release_date"
#     if re.search('d-link-reverse', ghtml):
#       note = re.search('d-link-reverse">(.*?)<', ghtml)
#     else:
#       note = 'not_scored_yet'
#     print(note)
#     if re.search('duration.*?>', ghtml):
#       duree = re.search('duration.*?>(.*?)<', ghtml, re.MULTILINE | re.DOTALL).group(1)
#       duree = re.sub(r"[\n\t\s]*", "", duree)
#     else:
#       duree = 'not_specified'

In [ ]:
print("Recherche des films dans les tops de sens critiques")
all_films = []
global all_films
i = 0
tmp_file = 1
for url in all_top_url:
    i += 1
    print('['+str(i)+'/'+str(len(all_top_url))+'] https://www.senscritique.com'+url)
    html = get_html('https://www.senscritique.com'+url, 3)
    
    if re.search('.*top/resultats.*', url):
        films_resultats(html)
    else:
        print(f'film top {url}')
        films_tops(html)

    if len(all_films) / 1000 > tmp_file:
        save_file('out_'+str(len(all_films))+'.csv', all_films)
        tmp_file += 1

print(str(len(all_films))+' films au total sur sens critique')
save_file('senscritique.csv', all_films)

[1/2243] https://www.senscritique.com/films/tops/top111
film top /films/tops/top111
111 new film sur 111 checked. TOTAL: 111
[2/2243] https://www.senscritique.com/films/tops/top100-des-top10
film top /films/tops/top100-des-top10
70 new film sur 100 checked. TOTAL: 181
[3/2243] https://www.senscritique.com/top/resultats/les_meilleurs_films_de_2022/3167299
30 new film sur 30 checked. TOTAL: 211
[4/2243] https://www.senscritique.com/top/resultats/les_meilleurs_films_de_2021/2917616
100 new film sur 100 checked. TOTAL: 311
[5/2243] https://www.senscritique.com/top/resultats/les_meilleurs_films_batman/1056032
22 new film sur 25 checked. TOTAL: 333
[6/2243] https://www.senscritique.com/top/resultats/les_meilleurs_films_de_2020/2582670
98 new film sur 100 checked. TOTAL: 431
[7/2243] https://www.senscritique.com/top/resultats/les_meilleurs_films_de_2019/2301802
96 new film sur 100 checked. TOTAL: 527
[8/2243] https://www.senscritique.com/top/resultats/les_meilleurs_films_d_animation_pixar/429